导入需要使用的包

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

设置超参数

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100

创建必要的文件夹

In [5]:
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

图像预处理pipeline，由于是灰度图，所以`transforms.Normalize(mean=[0.5], std=[0.5])`里面只有一个数，如果是RGB三通道的图，则应为：
~~~python
transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))])
~~~

In [6]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])])

加载数据并创建Loader

In [8]:
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

创建Discriminator和Generator

In [10]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

D = D.to(device)
G = G.to(device)

使用二元交叉熵损失分别作为Discriminator和Generator的损失函数，这里要注意的是有两部分损失分开的。二元交叉熵损失的计算方式为：
$$
BCE(a, y)= - y * \log(a) - (1-y) * \log(1 - a)
$$
其中$a$为模型输出，$y$为实际标签。

In [12]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

创建两个helper函数：

In [13]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

开始训练

In [ ]:
# 训练主loop
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # 创建后面给二元交叉熵损失的标签
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                           训练discriminator                        #
        # ================================================================== #

        # 计算二元损失函数，由于这里的real_labels是全1的，所以损失函数中的(1-y) * log(1 - D(x))都是0.
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # 利用fake_labels计算二元损失函数，由于是全零的，所以损失函数中y * log(D(x))都是0.
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 优化参数
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                             训练generator                          #
        # ================================================================== #

        # 通过随机输入生成的图片来计算生成器的loss
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        

        # 参照论文：https://arxiv.org/pdf/1406.2661.pdf中第三章，最大化 log(D(G(z))来训练G
        # 最大化 log(D(G(z))就是-log(D(G(z))的最小化
        # 所以用real_labels来计算损失
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # 保存图像
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # 保存生成的图像
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0570, g_loss: 4.5246, D(x): 0.99, D(G(z)): 0.05
Epoch [0/200], Step [400/600], d_loss: 0.3659, g_loss: 5.6716, D(x): 0.90, D(G(z)): 0.16
Epoch [0/200], Step [600/600], d_loss: 0.1519, g_loss: 5.2614, D(x): 0.96, D(G(z)): 0.07
Epoch [1/200], Step [200/600], d_loss: 0.0465, g_loss: 4.5738, D(x): 0.99, D(G(z)): 0.04
Epoch [1/200], Step [400/600], d_loss: 0.1414, g_loss: 3.9047, D(x): 0.93, D(G(z)): 0.05
Epoch [1/200], Step [600/600], d_loss: 0.2510, g_loss: 2.9919, D(x): 0.94, D(G(z)): 0.10
Epoch [2/200], Step [200/600], d_loss: 0.6416, g_loss: 3.6794, D(x): 0.86, D(G(z)): 0.23
Epoch [2/200], Step [400/600], d_loss: 0.5061, g_loss: 3.5212, D(x): 0.81, D(G(z)): 0.07
Epoch [2/200], Step [600/600], d_loss: 0.2967, g_loss: 2.8699, D(x): 0.90, D(G(z)): 0.11
Epoch [3/200], Step [200/600], d_loss: 0.1843, g_loss: 3.0504, D(x): 0.94, D(G(z)): 0.10
Epoch [3/200], Step [400/600], d_loss: 0.4663, g_loss: 2.6812, D(x): 0.87, D(G(z)): 0.23
Epoch [3/200], Step [

In [ ]:
保存模型，需要把D和G都保存下来。

In [ ]:
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')